### Задание

Реализуйте алгоритм классификации метод k ближайших соседей.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен
* Имплементация должна быть максимально векторизованной и, где это возможно, не использовать циклы

Необходимо реализовать класс KnnBruteClassifier, с реализацией прототипа, представленного ниже.

Должна быть реализована поддержка метрики расстояния L2 (параметр metric) и параметр weights типа 'uniform' и 'distance'.

В качестве входного файла необходимо использовать файл "knn_data_XXX.npy", полученный от бота командой /get seminar04

В качестве решения необходимо отправить боту, указав seminar04 в поле caption,  следующие файлы:
* knn.ipynb - содержит класс, реализующий ваш алгоритм
* results.npy - файл с результатами тестов, который можно будет сгенерировать с помощью этого ноутбука

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.neighbors.KNeighborsClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [28]:
import numpy as np

In [29]:
class KnnBruteClassifier(object):
    '''Классификатор реализует взвешенное голосование по ближайшим соседям. 
    Поиск ближайшего соседа осуществляется полным перебором.
    Параметры
    ----------
    n_neighbors : int, optional
        Число ближайших соседей, учитывающихся в голосовании
    weights : str, optional (default = 'uniform')
        веса, используемые в голосовании. Возможные значения:
        - 'uniform' : все веса равны.
        - 'distance' : веса обратно пропорциональны расстоянию до классифицируемого объекта
        -  функция, которая получает на вход массив расстояний и возвращает массив весов
    metric: функция подсчета расстояния (по умолчанию l2).
    '''
    def __init__(self, n_neighbors=1, weights='uniform', metric="l2"):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
     
    def fit(self, x, y):
        '''Обучение модели.
        Парметры
        ----------
        x : двумерным массив признаков размера n_queries x n_features
        y : массив/список правильных меток размера n_queries
        Выход
        -------
        Метод возвращает обученную модель
        '''
        self.x_train = x
        self.y_train = y
        
    def predict(self, x):
        """ Предсказание класса для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        y : Массив размера n_queries
        """
        distances, indices = self.kneighbors(x, self.n_neighbors)
        if self.weights == 'uniform':
            weights = np.ones(distances.shape)
        elif self.weights == 'distance':
            weights = 1 / distances
        y_train_subset = self.y_train[indices]
        
        if self.weights == 'uniform':
            mode = np.array([np.argmax(np.bincount(row)) for row in y_train_subset])
        elif self.weights == 'distance':
            mode = np.array([np.argmax(np.bincount(row, weights=weights[i])) for i, row in enumerate(y_train_subset)])
        return mode
        
    def predict_proba(self, X):
        """Возвращает вероятности классов для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        p : массив размера n_queries x n_classes] c вероятностями принадлежности 
        объекта к каждому классу
        """
        distances, indices = self.kneighbors(X, self.n_neighbors)
        if self.weights == 'uniform':
            weights = np.ones(distances.shape)
        elif self.weights == 'distance':
            weights = 1 / distances
        y_train_subset = self.y_train[indices]

        if self.weights == 'uniform':
            probabilities = np.array([np.bincount(row) / self.n_neighbors for row in y_train_subset])
        elif self.weights == 'distance':
            total_weights = np.sum(weights, axis=1)
            probabilities = np.array([np.bincount(row, weights=weights[i]) / total_weights[i] for i, row in enumerate(y_train_subset)])
        return probabilities
        
    def kneighbors(self, x, n_neighbors):
        """Возвращает n_neighbors ближайших соседей для всех входных объектов и расстояния до них
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        neigh_dist массив размера n_queries х n_neighbors
        расстояния до ближайших элементов
        neigh_indarray, массив размера n_queries x n_neighbors
        индексы ближайших элементов
        """
        if self.metric == 'l2':
            diff = x[:, :, np.newaxis] - self.x_train.T[np.newaxis, :, :]
            neigh_dist = np.sqrt(np.sum(diff ** 2, axis=1)) 
        neigh_indarray = np.argsort(neigh_dist)[:, :n_neighbors]
        neigh_dist = np.take_along_axis(neigh_dist, neigh_indarray, axis=1)
        return neigh_dist, neigh_indarray

In [30]:
def load_file(filename):
    """
    TODO: Необходимо загрузить файл задания и вернуть словарь с ключами "X_train", "X_test", "y_train"
    """
    data = np.load(filename, allow_pickle=True)
    X_train = data.item().get("X_train")
    X_test = data.item().get("X_test")
    y_train = data.item().get("y_train")
    data_dict = {"X_train": X_train, "X_test": X_test, "y_train": y_train }
    return data_dict

In [31]:
input_filename = "knn_data_041.npy" #TODO задать путь к входному файлу
data_dict = load_file(input_filename)

In [32]:
model = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_n5_y_predict = model.predict(data_dict["X_test"])

In [33]:
model = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_10_y_predict = model.predict(data_dict["X_test"])

In [34]:
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_distance_n5_y_predict = model.predict(data_dict["X_test"])

In [35]:
output_filename = "results.npy"
result_dict = {
    "input_filename": input_filename,
    "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
    "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
    "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
}
np.save(output_filename, result_dict)